<a href="https://colab.research.google.com/github/punkmic/Topic-Modeling-Reclame-Aqui/blob/master/Topic_Modeling_with_BERTopic_Reclame_aqui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Topic Modeling with BERTopic - Reclame Aqui**

BERTopic is a topic modeling technique that leverages transformers and a custom class-based TF-IDF to create dense clusters allowing for easily interpretable topics whilst keeping important words in the topic descriptions 

Reference: (https://maartengr.github.io/BERTopic/index.html).

### **Enabling the GPU**

We will use the GPU provided by COLAB to accelarate our model training. To enable GPUs for the notebook:
1- Navigate to Edit -> Notebook Settings
2- Select GPU from the Hardware Accelerator drop-down

In [129]:
# verify if GPU is enable
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Jan  6 20:30:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0    34W /  70W |   3908MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### **Setup**

In [130]:
%%capture
!pip install pyspellchecker
!pip install bertopic
!pip install kaleido # for save BERTopic plots as image

In [131]:
import pandas as pd # for data manipulation
import os # for interacting with the operating system
import nltk # for natural language processing
import string # for string manipulation
import re # for for regular expressions
import matplotlib.pyplot as plt # for visualization
import spacy # for lemmatize portuguese text
from bertopic import BERTopic # for topic modeling
from spellchecker import SpellChecker # for spell check

In [132]:
%%capture
# Install spacy pt_core_news_sm for portuguese text
!python -m spacy download pt_core_news_sm

In [133]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [134]:
# Download dataset with stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [135]:
# Donwload datasets for lemmatization
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [136]:
# Donwload dependency need to stem portuguese text
nltk.download('rslp')

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

### **Load data from [Github](https://github.com/punkmic/Topic-Modeling-Reclame-Aqui.git)**

In [137]:
#!git clone https://github.com/punkmic/Topic-Modeling-Reclame-Aqui.git

In [138]:
# Change directory
%cd /content/Topic-Modeling-Reclame-Aqui 

# Update files from remote repository
!git pull 

# Return to work directory
%cd ..

# Check current directory
!pwd

/content/Topic-Modeling-Reclame-Aqui
Already up to date.
/content
/content


In [139]:
def read_data(path_csv, drop_duplicates = True, lower=True):
 
  # use the read_csv method to read csv file
  df = pd.read_csv(path_csv)
  
  if drop_duplicates:
    # read and return the CSV file using the read_csv method
    print(f"Shape before remove duplicates: {df.shape}")

    # use the drop_duplicated method to drop duplicates rows
    df = df.drop_duplicates(subset="text")

    print(f"Shape after remove duplicates: {df.shape}")

    if lower:
      # apply the str.lower() method to each element in the dataframe
      df = df.applymap(str.lower)
    
     # rename columns
    df.columns = ["title", "documents"] 

    # use the replace() method to replace the string with an empty string
    df = df.replace(re.compile('\[editado pelo reclame aqui\]|editado pelo reclame aqui|Editado pelo Reclame Aqui'), '')
    df = df.replace(re.compile('\[casas bahia\]|Casa Bahia|Casas Bahia|casa bahia'), '')
    df = df.replace(re.compile('\[magazine luiza\]|Magazine luiza|Magazine Luiza| Magazine luizar|Magazine Luizar'), '')
    df = df.replace(re.compile('\[mercado livre\]|Mercado Livre|Mercado livre'), '')
    df = df.replace(re.compile('\[americana\]|Ameriacanas|ameriacanas'), '')

    # drop the old index column
    df.reset_index(inplace = True, drop = True)
  return df

### **Preprocessing**

#### **Tokenization**

Tokenization aims to breaking text down into its component parts

In [140]:
WORD_TOKENIZER = nltk.tokenize.word_tokenize
def tokenize(text):
  tokens = [token.strip().lower() for token in WORD_TOKENIZER(text, language="portuguese")]
 
  # set a pattern to detect patterns such as x x, xxx x, xxx xxx
  pattern = r"\b\w+\s+\w+\b"
 
  # filter tokens by pattern
  filtered_words = [word for word in tokens if re.search(pattern, word)]

  # return token if not in filter list
  return [token for token in tokens if token not in filtered_words]

#### **Stem** 

Stem the tokens. This step aims to remove morphological affixes and normalize to standardized stem forms

In [141]:
STEMMER = nltk.stem.RSLPStemmer()
def stem(tokens):
  return [STEMMER.stem(token) for token in tokens]

#### **Lemmatize**

Lemmatize the tokens. Retains more natural forms than stemming, but assumes all tokens nons unless tokens are passed as (word, pos) tuples. Note: nltk lemmatize does not suport portugues language

In [142]:
LEMMATIZER = nltk.WordNetLemmatizer()

def lemmatize(tokens):
  lemmas = []
  for token in tokens:
      if isinstance(token, str):
          # treats token like a noun
          lemmas.append(LEMMATIZER.lemmatize(token)) 
      else: 
          # assume a tuple of (word, pos)
          lemmas.append(LEMMATIZER.lemmatize(*token))
  return lemmas

**Lemmatize option for portuguese text**

In [143]:
# load portuguese model
nlp = spacy.load('pt_core_news_sm')

def lemmatize_pt(tokens):

  # Create a spaCy Doc object and apply the lemmatization
  doc = nlp(' '.join(tokens))

  # Return lemmatize
  return [token.lemma_ for token in doc]

#### **Remove stopwords**

Stop words are things like articles and conjunctions that usually do not offer a lot of value in an analysis.

In [144]:
def remove_stopwords(tokens, stopwords=None, custom_stop_words = None):

  if custom_stop_words is None:
    custom_stop_words = ['amazon', 'americanas', 'casas bahia', 'magazine luiza', 'shein', 'kabum',
                       'samsung', 'mercado livre', 'banco brasil', 'apple', 'magazine', 'luiza', 'luizar',
                      'casas', 'bahia', 'casa', 'mercado', 'livre']

  # Use the default stop words if none is passed
  if stopwords is None:
    stopwords = nltk.corpus.stopwords.words('portuguese') + custom_stop_words
  
  # Filter the list of tokens to exclude the stop word tokens
  return [token for token in tokens if token not in stopwords]

In [145]:
custom_stop_words = ['amazon', 'americanas', 'casas bahia', 'magazine luiza', 'shein', 'kabum',
                       'samsung', 'mercado livre', 'banco brasil', 'apple', 'magazine', 'luiza', 'luizar',
                      'casas', 'bahia', 'casa', 'mercado', 'livre']
assert remove_stopwords(['compra', 'echar', 'em esse', 'amazon', 'pude'], custom_stop_words=custom_stop_words) == ['compra', 'echar', 'em esse', 'pude']

#### **Remove hyperlinks**

Removes http/s links from the tokens.

In [146]:
def remove_links(tokens):
  # Filter tokens that starts with "http://" or "https://"
  return [token for token in tokens 
          if not token.startswith("http://")
          and not token.startswith("https://")]

In [147]:
assert remove_links(['bom', 'http://online', 'https://offline']) == ['bom']

#### **Remove numbers**

In [148]:
def remove_numbers(tokens):
  # Use a regular expression to match words that contain numbers
  pattern = r"\b\w*\d\w*\b"
  tokens = [token for token in tokens if not re.sub(pattern, "", token) == ""]
  
  # Filter out number tokens using a list comprehension and the isnumeric method
  return [token for token in tokens if not token.isnumeric()]

In [149]:
assert remove_numbers(['ola', 'bicicleta', '1', '2002']) == ['ola', 'bicicleta']

#### **Remove date**

In [150]:
def remove_date(tokens):
  # Compile a regular expression to match dates in the format dd/mm or dd/mm/yyyy
  date_regex = re.compile(r'\d{2}/\d{2}(/\d{4})?')

  # Use the regex to find all the tokens that match the date pattern
  dates = [token for token in tokens if date_regex.fullmatch(token)]

  # Filter the list of tokens to exclude the date tokens
  filtered_tokens = [token for token in tokens if token not in dates]

  # Return the filtered tokens
  return filtered_tokens

In [151]:
assert remove_date(['texto', 'data', '20/10', 'seguro', '02/09/2014']) == ['texto', 'data', 'seguro']

#### **Remove punctuation**

In [152]:
def remove_punctuation(tokens,
                       strip_mentions=True,
                       strip_hashtags=True):

    tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens]

    # Filter punctuation tokens
    tokens = [token.strip() for token in tokens if token not in string.punctuation]

    # Remove @ symbol from left side of tokens
    if strip_mentions:
        tokens = [t.lstrip(r"([!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~])\1+") for t in tokens]

    # Remove # symbol from left side of tokens
    if strip_hashtags:
        tokens = [t.lstrip(r"([!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~])\1+") for t in tokens]

    return tokens

In [153]:
assert remove_punctuation(['limpo', 'acento/  ///', 'simples???', 'onde', ',']) == ['limpo', 'acento', 'simples', 'onde']

#### **Remove short tokens**

In [154]:
def remove_short_tokens(tokens):
  # Filter the list of tokens to exclude tokens that are shorter than four letters
  filtered_tokens = [token for token in tokens if len(token) >= 4]

  # Return the filtered tokens
  return filtered_tokens

In [155]:
assert remove_short_tokens(['sair', 'um', 'correto', 'igual', 'oi', 'de', 'em']) == ['sair', 'correto', 'igual']

#### **Correction of spelling errors**

In [156]:
# Create a SpellChecker object
spell = SpellChecker(language='pt')

def check_spell_errors(text):
  result = []
  for token in text:
    # Correct the spelling errors in the text
    corrected_text = spell.correction(token)

    # If no correction is present user the original text
    if corrected_text == None:
      corrected_text =  token
  
    result.append(corrected_text)
  # Return the corrected text
  return result

#### **Remove extra white spaces**

In [157]:
def remove_whitespace(document):
    return  " ".join(document.split())

In [158]:
def preprocess(df, colname):
  df[colname]= df[colname].str.lower()
  df[colname]= df[colname].apply(remove_whitespace)
  df['tokenize'] = df[colname].apply(tokenize)
  #df['spell_error'] = df['tokenize'].apply(check_spell_errors)
  df['remove_links'] = df['tokenize'].apply(remove_links)
  df['remove_punctuation'] = df['remove_links'].apply(remove_punctuation)
  df['remove_numbers'] = df['remove_punctuation'].apply(remove_numbers)
  df['remove_date'] = df['remove_numbers'].apply(remove_date)
  df['remove_short_tokens'] = df['remove_date'].apply(remove_short_tokens)
  df['remove_stop_words'] = df['remove_short_tokens'].apply(remove_stopwords)
  df['lemmas'] = df['remove_stop_words'].apply(lemmatize_pt) 
  df['corpus'] = df['lemmas'].apply(lambda x: ' '.join(x))
  return df

In [189]:
def get_data(features):
  dfs = {}
  # create models for documents and title features
  for feature in features:
    df = None
    if not os.path.exists(f'/content/Topic-Modeling-Reclame-Aqui/results/dataset/preprocessed/table_{feature}.csv'):

      path_csv = "/content/Topic-Modeling-Reclame-Aqui/corpus.csv"
  
      if df is None:
        # read data 
        df = read_data(path_csv)
  
      # preprocess data
      df = preprocess(df, feature)

      # Set the path to save 
      path = '/content/Topic-Modeling-Reclame-Aqui/results/dataset/preprocessed/'

      # Use makedirs() to create a new directory if it does not exists
      if not os.path.exists(path):
        os.makedirs(path)

      # Save the DataFrame to a CSV file
      df.to_csv(path + f'table_{feature}.csv')
      print(f'Dataset saved into {path} directory.')
  for feature in features:
    df = pd.read_csv(f'/content/Topic-Modeling-Reclame-Aqui/results/dataset/preprocessed/table_{feature}.csv')['corpus']
    dfs[feature] = df
  return dfs

In [199]:
features=['title', 'documents']
dfs = get_data(features)
print(dfs.keys())

dict_keys(['title', 'documents'])


In [201]:
assert list(dfs.keys()) == features

In [198]:
DATA_FRAME_FEATURE_NAME = 'title'

corpus = dfs[DATA_FRAME_FEATURE_NAME]
corpus[0]

'pedir cancelado justificativa após semana compro'

## **Training a BERTopic Model**

The BERTopic algorithm has several advantages over other topic modeling algorithms. It is able to handle sparse data, it is scalable to large datasets, and it is able to learn topics that are not well-defined or are overlapping.

As our data language is portuguese we will going to set language to multilingual.

Create a new BERTopic model and train it. By default BERTopic use the paraphrase-multilingual-MiniLM-L12-v2 model for multi language documents. For others model check here [BERTopic sentence transformers](https://maartengr.github.io/BERTopic/getting_started/embeddings/embeddings.html#sentence-transformers)

In [161]:
# Create a new BERTopic model using multilingual option
topic_model = BERTopic(language="multilingual", calculate_probabilities=True, verbose=True)

# Train model 
topics, probs = topic_model.fit_transform(corpus)

Batches:   0%|          | 0/329 [00:00<?, ?it/s]

2023-01-06 20:47:25,291 - BERTopic - Transformed documents to Embeddings
2023-01-06 20:47:48,727 - BERTopic - Reduced dimensionality
2023-01-06 20:47:54,191 - BERTopic - Clustered reduced embeddings


BERTopic works in three main steps: 


1.   Documents are first converted to numeric data. It extracts different embeddings based on the context of the word. For this, a sentence transformation model is used.
2.  Documents with similar topics are then grouped together forming clusters with similar topics. For this purpose, BERTopic uses the clustering algorithm UMAP to lower the dimensionality of the embeddings. Then the documents are clustered with the density-based algorithm HDBSCAN.
3. BERTopic extracts topics from clusters using a class-based TF-IDF score. This score gives the importance of each word in a cluster. Topics are then created based on the most important words measured by their C-TF-IDF score.

For more information check this link [BERTopic](https://towardsdatascience.com/topic-modeling-with-bert-779f7db187e6)



### **Extracting Topics**

In [162]:
# Print the most frequent topics
freq = topic_model.get_topic_info()

# Show the top 5 most frequent topics
freq.head(5)

,Topic,Count,Name
0,-1,5119,-1_entregar_produto_pedir_compra
1,0,633,0_cartão_crédito_cobrar_cobrança
2,1,502,1_celular_aparelho_smartphone_comprar
3,2,257,2_whatsapp_contato_entregar_atendimento
4,3,245,3_brasil_brasileiro_chegar_pedir


The table above shows the five most freqeuente topics and the words present on it extract by BERTopic. -1 refers to all outliers and should be ignored.

In [163]:
# show the most frequent topic
topic_model.get_topic(0)

[('cartão', 0.05457878492990092),
 ('crédito', 0.02861950568175978),
 ('cobrar', 0.018609249540710374),
 ('cobrança', 0.017638062155466563),
 ('fatura', 0.017625068530599128),
 ('compra', 0.016799195872435214),
 ('valor', 0.01647283379221534),
 ('assinatura', 0.014099442492386203),
 ('pagamento', 0.013904072337887444),
 ('contar', 0.013508000050436772)]

**Note:** BERTopic is stocastich which means that the topics might differ across runs this is mostly due to the stocastisch nature of UMAP

#### **Save topic info table as CSV**

In [164]:
# Set the path to save 
path = f'/content/Topic-Modeling-Reclame-Aqui/results/{DATA_FRAME_FEATURE_NAME}/topic_info_tables/'

# Use makedirs() to create a new directory if it does not exists
if not os.path.exists(path):
  os.makedirs(path)

# Save table as csv
freq.head(10).to_csv(path + 'topic_info_preprocessed_lemma.csv')

## **Visualization**

### **Intertopic Distance Map**

This graph shows the distance intertopic and help us understand the promixity of topics

In [165]:
fig = topic_model.visualize_topics(width=800, height=800)
fig

#### **Save intertopic distance map**

In [166]:
# Set the path to save 
path = f'/content/Topic-Modeling-Reclame-Aqui/results/{DATA_FRAME_FEATURE_NAME}/intertopic_distance_map/'

# Use makedirs() to create a new directory if it does not exists
if not os.path.exists(path):
  os.makedirs(path)


fig.write_image(path + "idm_preprocessed_lemma.png", format="png")
fig.write_html(path + "idm_preprocessed_lemma.html")

[Github link](https://github.com/punkmic/Topic-Modeling-Reclame-Aqui/blob/master/results/intertopic_distance_map/idm_preprocessed_lemma.png?raw=true)

### **Visualize Topic Hierarchy**

The topics that were created can be hierarchically reduced. This visualization shows how the topics relate to one another.

In [167]:
fig = topic_model.visualize_hierarchy(top_n_topics=30, width=800, height=800)
fig

#### **Save Hierarchical Clustering**

In [168]:
# Set the path to save 
path = f'/content/Topic-Modeling-Reclame-Aqui/results/{DATA_FRAME_FEATURE_NAME}/hierarchical_clustering/'

# Use makedirs() to create a new directory if it does not exists
if not os.path.exists(path):
  os.makedirs(path)


fig.write_image(path + "hc_preprocessed_lemma.png", format="png")

[Github link](https://github.com/punkmic/Topic-Modeling-Reclame-Aqui/blob/master/results/hierarchical_clustering/hc_preprocessed_lemma.png?raw=true)

### **Visualize Terms**

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation.

In [169]:
fig = topic_model.visualize_barchart(top_n_topics=12, width=300, height=300)
fig

#### **Save Top Word Scores Bar Chart**

In [170]:
# Set the path to save 
path = f'/content/Topic-Modeling-Reclame-Aqui/results/{DATA_FRAME_FEATURE_NAME}/top_words_scores/'

# Use makedirs() to create a new directory if it does not exists
if not os.path.exists(path):
  os.makedirs(path)


fig.write_image(path + "tws_preprocessed_lemma.png", format="png")

[Github link](https://github.com/punkmic/Topic-Modeling-Reclame-Aqui/blob/master/results/top_words_scores/tws_preprocessed_lemma.png?raw=true)

### **Visualize Topic Similarity**

This plot shows a similarity matrix by simply applying cosine similarities through those topic embeddings generate by BERTopic through both c-TF-IDF and embeddings. This matrix indicate how similar certain topics are to each other.

In [171]:
fig = topic_model.visualize_heatmap(n_clusters=20, width=800, height=800)
fig

#### **Save Similarity Matrix**

In [172]:
# Set the path to save 
path = f'/content/Topic-Modeling-Reclame-Aqui/results/{DATA_FRAME_FEATURE_NAME}/similarity_matrix/'

# Use makedirs() to create a new directory if it does not exists
if not os.path.exists(path):
  os.makedirs(path)


fig.write_image(path + "sm_preprocessed_lemma.png", format="png")

[Github link](https://github.com/punkmic/Topic-Modeling-Reclame-Aqui/blob/master/results/similarity_matrix/sm_preprocessed_lemma.png?raw=true)

### **Visualize Term Score Decline**

Topics are represented by a number of words starting with the best representative word. Each word is represented by a c-TF-IDF score. The higher the score, the more representative a word to the topic is. Since the topic words are sorted by their c-TF-IDF score, the scores slowly decline with each word that is added.

In [173]:
fig = topic_model.visualize_term_rank()
fig

#### **Save Term score decline per Topic**

In [174]:
# Set the path to save 
path = f'/content/Topic-Modeling-Reclame-Aqui/results/{DATA_FRAME_FEATURE_NAME}/term_score_decline_topic/'

# Use makedirs() to create a new directory if it does not exists
if not os.path.exists(path):
  os.makedirs(path)


fig.write_image(path + "tsdp_preprocessed_lemma.png", format="png")

[Github link](https://github.com/punkmic/Topic-Modeling-Reclame-Aqui/blob/master/results/term_socore_decline_topic/tsdp_preprocessed_lemma.png?raw=true)

### **Term search**

In [175]:
# Find topics that contains blackfriday term
similar_topics, similarity = topic_model.find_topics("blackfriday", top_n=5)

# Show similar topics
similar_topics

[12, 72, 29, 34, 69]

In [176]:
# Show a specific topic
topic_model.get_topic(64)

[('chat', 0.04389444342724184),
 ('desbloqueio', 0.028855653461456767),
 ('autoatendimento', 0.027554047823656808),
 ('preciso', 0.025309334157012094),
 ('insatisfação', 0.02478232909919437),
 ('serviço', 0.02159723247698404),
 ('sugirar', 0.02150073700320711),
 ('falar', 0.02085206565546488),
 ('batepapo', 0.020044331853462258),
 ('atendimento', 0.019160721090437746)]

### **Hiperparameter optimization**

In [177]:
%%capture
!pip install optuna
!pip install hdbscan
!pip install umap-learn

In [178]:
import json
import optuna # for hyperparameter optimization
from hdbscan import HDBSCAN # for clustering
from umap import UMAP # for dimension reduction
from sklearn.feature_extraction.text import CountVectorizer # for convert text documents to matrix of tokens count
from bertopic.vectorizers import ClassTfidfTransformer 
import ast # for convert str to tuple

In [179]:
def save_documents(model, corpus, label, reduced):
  fig = topic_model.visualize_documents(corpus, hide_document_hover=True, hide_annotations=True, width=800, height=800)

  # Set the path to save 
  path = f'/content/Topic-Modeling-Reclame-Aqui/results/{DATA_FRAME_FEATURE_NAME}/documents_n_topics/'

  if reduced:
    path = path + 'reduced/'

  # Use makedirs() to create a new directory if it does not exists
  if not os.path.exists(path):
    os.makedirs(path)

  fig.write_image(path + f"document_n_topics_trial_{label}.png", format="png")
  fig.write_html(path + f"document_n_topics_trial_{label}.html")

In [180]:
def save_topics(model, label, reduced=False):
  fig = model.visualize_topics(width=800, height=800)

  # Set the path to save 
  path = f'/content/Topic-Modeling-Reclame-Aqui/results/{DATA_FRAME_FEATURE_NAME}/intertopic_distance_map/'

  if reduced:
    path = path + 'reduced/'

  # Use makedirs() to create a new directory if it does not exists
  if not os.path.exists(path):
    os.makedirs(path)

  fig.write_image(path + f"intertopic_distance_map_trial_{label}.png", format="png")
  fig.write_html(path + f"intertopic_distance_map_trial_{label}.html")

In [181]:
def save_hierarchy(model, label, reduced=False):
  fig = model.visualize_hierarchy(width=800, height=800)

  # Set the path to save 
  path = f'/content/Topic-Modeling-Reclame-Aqui/results/{DATA_FRAME_FEATURE_NAME}/hierarchical_clustering/'

  if reduced:
    path = path + 'reduced/'

  # Use makedirs() to create a new directory if it does not exists
  if not os.path.exists(path):
    os.makedirs(path)

  fig.write_image(path + f"hierarchical_clustering_trial_{label}.png", format="png")
  fig.write_html(path + f"hierarchical_clustering_trial_{label}.html")

In [182]:
def save_top_words_scores(model, label, reduced=False):
  fig = model.visualize_barchart(top_n_topics=12, width=300, height=300)

  # Set the path to save 
  path = f'/content/Topic-Modeling-Reclame-Aqui/results/{DATA_FRAME_FEATURE_NAME}/top_words_scores/'

  if reduced:
    path = path + 'reduced/'

  # Use makedirs() to create a new directory if it does not exists
  if not os.path.exists(path):
    os.makedirs(path)

  fig.write_image(path + f"top_words_scores_trial_{label}.png", format="png")
  fig.write_html(path + f"top_words_scores_trial_{label}.html")

In [183]:
def save_similarity_matrix(model, label, reduced=False):
  fig = model.visualize_heatmap()

  # Set the path to save 
  path = f'/content/Topic-Modeling-Reclame-Aqui/results/{DATA_FRAME_FEATURE_NAME}/similarity_matrix/'

  if reduced:
    path = path + 'reduced/'

  # Use makedirs() to create a new directory if it does not exists
  if not os.path.exists(path):
    os.makedirs(path)

  fig.write_image(path + f"similarity_matrix_trial_{label}.png", format="png")
  fig.write_html(path + f"similarity_matrix_trial_{label}.html")

In [184]:
def save_term_rank(model, label, reduced=False):
  fig = model.visualize_term_rank(width=800, height=800)

  # Set the path to save 
  path = f'/content/Topic-Modeling-Reclame-Aqui/results/{DATA_FRAME_FEATURE_NAME}/term_score_decline_topic/'

  if reduced:
    path = path + 'reduced/'

  # Use makedirs() to create a new directory if it does not exists
  if not os.path.exists(path):
    os.makedirs(path)

  fig.write_image(path + f"term_score_trial_{label}.png", format="png")
  fig.write_html(path + f"term_score_trial_{label}.html")

In [185]:
def save_hyperparameters(trial_params, label):
  # Set the path to save 
  path = f'/content/Topic-Modeling-Reclame-Aqui/results/{DATA_FRAME_FEATURE_NAME}/hyperparameters/'

  # Use makedirs() to create a new directory if it does not exists
  if not os.path.exists(path):
    os.makedirs(path)

  with open(path + f"hyperparameters_trial_{label}.json", "w") as f:
    f.write(json.dumps(trial_params))


In [186]:
def save_freq_topics(model, label):
   # Print the most frequent topics
  freq = topic_model.get_topic_info()

  # Show the top 5 most frequent topics
  freq = freq.head(5)

  # Set the path to save 
  path = f'/content/Topic-Modeling-Reclame-Aqui/results/{DATA_FRAME_FEATURE_NAME}/frequent_topics/' 
  freq.to_html(path + f"freq_topics_trial_{label}.json")
  freq.to_csv(path + f"freq_topics_trial_{label}.csv")

In [187]:
def optimizer(trial):

  # BERTopic hyperparameters
  top_n_words = trial.suggest_int('bertopic__top_n_words', 5, 14)
  n_gram_range = ast.literal_eval(trial.suggest_categorical('bertopic__n_gram_range', ['(1,1)', '(1,2)', '(1,3)']))
  min_topic_size = trial.suggest_int('bertopic__min_topic_size', 15, 20)
  diversity = trial.suggest_float('bertopic__diversity', 0.1, 0.9)
  outlier_threshold = trial.suggest_float('bertopic__outliers_threshold', 0.04, 0.09)
  nr_topics = trial.suggest_int('bertopic__nr_topics', 12, 20)

  # UMAP hyperparameters
  n_neighbors = trial.suggest_int('umap__n_neighbors', 10, 20)
  n_components = trial.suggest_int('umap__n_components', 2, 6)
  metric = trial.suggest_categorical('umap__metric', ['cosine', 'euclidean'])
  min_dist = trial.suggest_float('umap__min_dist', 0.1, 1.0)
  spread = trial.suggest_float('umap__spread', 0.1, 1.0)

  # HDBSCAN hyperprarameters
  min_cluster_size = trial.suggest_int('hdbscan__min_cluster_size', 10, 20)
  min_samples = trial.suggest_int('hdbscan__min_samples', 10, 20)
  cluster_selection_epsilon = trial.suggest_float('hdbscan__cluster_selection_epsilon', 0.1, 1.0)

  # CountVectorizer hyperparameters 
  max_features = trial.suggest_int('vectorizer__max_features', 3000, 6000)

  # reduce the impact of frequent words.
  ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

  # list of portuguese stop words + custom words
  stop_words = nltk.corpus.stopwords.words('portuguese') + custom_stop_words

  # create a new CountVectorizer to create a matrix of tokens count
  vectorizer_model = CountVectorizer(stop_words=stop_words, ngram_range=n_gram_range, max_features=max_features)

  # create a new UMAP model to reduce dimension
  umap_model = UMAP(n_neighbors=n_neighbors, metric=metric, random_state=42)

  # create a new HDBSCAN model to cluster documents
  hdbscan_model = HDBSCAN(min_cluster_size=min_cluster_size, prediction_data=True)

  # create a new BERTopic model using multilingual option
  model = BERTopic(language="multilingual", 
                         calculate_probabilities=True, 
                         verbose=False,
                         nr_topics='auto',
                         top_n_words=top_n_words,
                         n_gram_range=n_gram_range,
                         min_topic_size=min_topic_size,
                         diversity=diversity,
                         umap_model=umap_model,
                         hdbscan_model=hdbscan_model,
                         vectorizer_model=vectorizer_model,
                         ctfidf_model=ctfidf_model)
  
  label = trial.number
  params = trial.params

  # train BERTopic model 
  topics, probs = model.fit_transform(corpus)

  # save plots
  save_topics(model, label, False)
  save_documents(model, corpus, label, False)
  save_hierarchy(model, label, False)
  ##save_similarity_matrix(model, label, False)
  save_term_rank(model, label, False)
  save_top_words_scores(model, label, False)

  # reduce outliers
  new_topics = model.reduce_outliers(corpus,
                                     topics, 
                                     probabilities=probs,
                                     strategy="probabilities",
                                     threshold=outlier_threshold)
  
  # update model topics after outlier reduction
  model.update_topics(corpus, topics=new_topics)

  # save plots after reducion
  save_topics(model, label, True)
  save_documents(model, corpus, label, True)
  save_hierarchy(model, label, True)
  ##save_similarity_matrix(model, label, True)
  save_term_rank(model, label, True)
  save_top_words_scores(model, label, True)

  # save hyperparameters
  save_hyperparameters(params, label)

  # save reduction result
  save_freq_topics(model, label)

  # save model
  path = f'/content/Topic-Modeling-Reclame-Aqui/results/{DATA_FRAME_FEATURE_NAME}/models/'

  # Use makedirs() to create a new directory if it does not exists
  if not os.path.exists(path):
    os.makedirs(path)

  model.save(path + f"model_trial_{label}")

  return 0

In [89]:
%%time
# define the number of models to generate by optuna
NUMBER_OF_MODELS = 15

# create a new study
study = optuna.create_study(study_name='Bertopic')

# run the optmize function 
study.optimize(optimizer, n_trials=NUMBER_OF_MODELS)

[I 2023-01-06 19:21:56,054] A new study created in memory with name: Bertopic
2023-01-06 19:22:15,230 - BERTopic - Transformed documents to Embeddings
2023-01-06 19:22:35,132 - BERTopic - Reduced dimensionality
2023-01-06 19:22:37,271 - BERTopic - Clustered reduced embeddings
2023-01-06 19:22:42,540 - BERTopic - Reduced number of topics from 14 to 14
[I 2023-01-06 19:23:54,203] Trial 0 finished with value: 0.0 and parameters: {'bertopic__top_n_words': 5, 'bertopic__n_gram_range': '(1,1)', 'bertopic__min_topic_size': 15, 'bertopic__diversity': 0.746830833426787, 'bertopic__outliers_threshold': 0.06411398360593415, 'bertopic__nr_topics': 14, 'umap__n_neighbors': 15, 'umap__n_components': 6, 'umap__metric': 'euclidean', 'umap__min_dist': 0.25133308240297303, 'umap__spread': 0.11373918912306409, 'hdbscan__min_cluster_size': 20, 'hdbscan__min_samples': 12, 'hdbscan__cluster_selection_epsilon': 0.11754072133120577, 'vectorizer__max_features': 4868}. Best is trial 0 with value: 0.0.
2023-01-0

KeyboardInterrupt: ignored

#### **Merge of similar topics**

In [ ]:
# TODO: select the best model and merge similar topic and save final model

#topic_model = BERTopic.load("best_model")

In [ ]:
##########33!rm -rf /content/Topic-Modeling-Reclame-Aqui/results

In [3]:
! ssh-keygen -t rsa -b 4096
# Add github.com to our known hosts
! ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts
# Restrict the key permissions, or else SSH will complain.
! chmod go-rwx /root/.ssh/id_rsa

Generating public/private rsa key pair.
Enter file in which to save the key (/root/.ssh/id_rsa): 
/root/.ssh/id_rsa already exists.
Overwrite (y/n)? Y
Enter passphrase (empty for no passphrase): 
Enter same passphrase again: 
Your identification has been saved in /root/.ssh/id_rsa.
Your public key has been saved in /root/.ssh/id_rsa.pub.
The key fingerprint is:
SHA256:8+eNknI/3SRVFqCNFQI1Sx4LQRmHyWsxaWXIjLBgnR8 root@66134f0b36cb
The key's randomart image is:
+---[RSA 4096]----+
|   o..o **%X +o..|
|  . .o.E &* @   o|
|     .. o +* . ..|
|       . o      .|
|        S      . |
|         o    . .|
|          .... + |
|        . +o.o. .|
|         o o+..  |
+----[SHA256]-----+
# github.com:22 SSH-2.0-babeld-4ce3b487


In [4]:
! cat /root/.ssh/id_rsa.pub

ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQDEQ/953oQTI25HaIZyMi5vtWXXP0qy9R4lq10XSJrgU9SW+6gEML3a5Wfl8wfeKxMBTMo0ICJC86ermSQWhMg6pyeZvRin9R8MwVJxyUfbFCMkwmMnSV9JxtJpg6TpXp8qzJ304r+9SyauYjo6QcDBXAC1H7KhSCTiSAZWJuuf5+xvvzPFbZ4Zp4kyFOKzvDiFnTiTRs6WRKW9liQ7kguDOK67PUBmTGsgGweAZOPH2jI86Jkg8G4m2565fgGOznVUuPVxRoL0KCcflnOGykSCJez7gRUMvqCy6uy9cTkwHSTDSvvx/CxYQ3AOkioqQ+x6ehCrGRxw+ICKhtWuhblGQKLlHXcgKUda7cvQo4mPb3Y19WdtXwdUD5Gq/GWRcxBBs3Qg8SaVHEboeFQaehPIumenGf+xQ1LB39oGBkbDOWKSZnOs9PA0lro7qbCLzNQkjmcmM9R9rDxv2GaeX2Jo1uqSYiNRFveP2rPbQEPVvXBEMDo84FEkmQ84msEvOUENRSlJyOCQlCh+a3u71JgT8jXUhqFH9FukTHRyygQk8UggdlnW/BSqaGRuzjr6UVjBN+wiGMmwvSLFwPUOdCDC3Vy1HuMXcjR1vZe6i4ogMtR0+vYTJv4dZOwNg5+ZiJR4eVCQCZB7sc917gwiB3ytd7ISjaviQgbBp6HfhP+jqQ== root@66134f0b36cb


In [5]:
!git config --global user.email "mattheus_ribeiro@outlook.com"
!git config --global user.name "punkmic"

In [6]:
!ssh -T git@github.com

Hi punkmic! You've successfully authenticated, but GitHub does not provide shell access.


In [7]:
!git clone git@github.com:punkmic/Topic-Modeling-Reclame-Aqui.git

Cloning into 'Topic-Modeling-Reclame-Aqui'...
remote: Enumerating objects: 16267, done.
remote: Counting objects: 100% (2086/2086), done.
remote: Compressing objects: 100% (1818/1818), done.
remote: Total 16267 (delta 293), reused 1951 (delta 234), pack-reused 14181
Receiving objects: 100% (16267/16267), 126.63 MiB | 7.41 MiB/s, done.
Resolving deltas: 100% (1441/1441), done.
Checking out files: 100% (14808/14808), done.


In [ ]:
%cd /content/Topic-Modeling-Reclame-Aqui/

/content/Topic-Modeling-Reclame-Aqui


In [ ]:
!git add ./results/dataset/preprocessed/table.csv

In [ ]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	new file:   results/dataset/preprocessed/table.csv



In [ ]:
!git commit -m "Adding preprocessed dataset"

[master 30c507c] Adding preprocessed dataset
 1 file changed, 10511 insertions(+)
 create mode 100644 results/dataset/preprocessed/table.csv


In [ ]:
!git push origin master

Counting objects: 6, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (5/5), done.
Writing objects: 100% (6/6), 8.25 MiB | 1.45 MiB/s, done.
Total 6 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
remote: warning: See http://git.io/iEPt8g for more information.
remote: warning: File results/dataset/preprocessed/table.csv is 68.94 MB; this is larger than GitHub's recommended maximum file size of 50.00 MB
remote: warning: GH001: Large files detected. You may want to try Git Large File Storage - https://git-lfs.github.com.
To github.com:punkmic/Topic-Modeling-Reclame-Aqui.git
   c9fb74b..30c507c  master -> master
